### Importing libraries

In [2]:
import numpy as np
import pandas as pd
import sklearn
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import scipy
from datasets import load_dataset
from pydub import AudioSegment
import librosa

General Workflow:




1. Speech signal ---> Text




2. Text ----> Dirichlet Clusters




3. Dirichlet Cluster keywords ---> Search Engine / Youtube API







In [3]:
### Loading model to device

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!


In [4]:
sample_audio_path = "cmu_computer_graphics_course_admin_recording.mp3"
# audio = scipy.io.wavfile.read(mono_audio_path)[1]
# print(audio)
# print(audio.dtype)

### We use the librosa package here to read mp3 files, for more information regarding the librosa package, you can visit: https://github.com/librosa/librosa
Note: Librosa works for mp3, m4a files

In [5]:
y, sr = librosa.load(sample_audio_path)
print(y)
print(sr)

[1.9852335e-22 8.2718061e-24 1.9852335e-23 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00]
22050


In [6]:
output_result = pipe(y, generate_kwargs={"language": "english", "task": "transcribe"})
print(output_result["text"])

 Welcome to Computer Graphics 15462-662 at Carnegie Mellon University. I'm Kenan Crane. I'm a professor of computer science and robotics. And I also do research in computer graphics, so specifically in the area of geometric algorithms. The purpose of this video is to give you all the information that you'll need to succeed this semester. So periodically we'll upload little videos to cover administrative things, to talk about what's been going on this week, and to answer any significant questions that have come up. I should also say that all the information today is available on the course webpage at 15462.courses.cs.cmu.edu. So please go ahead, check out that link, read through especially the course info page in detail because there's a lot of things that I won't say here in this video but that are important for you to know as you go through the course. we have a great set of TAs this semester. So if you have any questions, please at any time, feel free to email them, email me, post a 

### Exporting the transcribed voice into a text file

In [7]:
output_file = "cmu_computer_graphics_intro_voice_transcribed.txt"
transcribed_text = output_result["text"]
with open(output_file, "w") as dst:
    dst.write(transcribed_text)